# Processing of a list of images

Select a result folder with the list of image files to process.

In [ ]:
import yaml
from pathlib import Path
from ipyfilechooser import FileChooser

srcdir, dstdir = "", ""
if Path("config.yml").exists():
    with open("config.yml", "r") as file:
        config = yaml.safe_load(file)
        if "source" in config.keys():
            srcdir = Path(config["source"])
        if "destination" in config.keys():
            dstdir = Path(config["destination"])

fc = FileChooser(dstdir, select_desc="Destination")
display(fc)

Load the list of files.

In [ ]:
import pandas as pd
dstdir = Path(fc.selected) if fc.selected is not None else Path(dstdir)
filelistname = dstdir / "filelist.csv"
print("Destination folder :", dstdir)
print("Output file list :", filelistname)
filelist = pd.read_csv(filelistname)
filelist

Process all files

In [ ]:
import octofish
from cellpose import models, core

model = models.Cellpose(gpu=core.use_gpu(), model_type="nuclei")
resolution_level = 1
crop = False
dummy_run = False

for row in filelist.iloc:
    if octofish.get_files(dstdir, row, "measurements").exists() is False:
        if octofish.get_files(dstdir, row, "regions").exists() is True:            
            octofish.process_item(row, resolution_level, model, dstdir, crop, dummy_run)